### 下面是对[nanoGPT](https://github.com/karpathy/nanoGPT)代码的解读
[nanoGPT](https://github.com/karpathy/nanoGPT)是用于训练/微调中型 GPT 的最简单、最快速的仓库。`train.py` 在 `OpenWebText` 上重现了 `GPT-2（124M）`，在单个 `8XA100 40GB` 节点上大约经过 `4` 天的训练。代码本身简单易读：`train.py` 是一个基本训练循环，`model.py` 是一个 GPT 模型定义，可以选择从 OpenAI 加载 GPT-2 权重。



# GPT实现

In [1]:
# 导包
import os
import time
import math
import pickle
from contextlib import nullcontext
import numpy as np
import inspect
from dataclasses import dataclass

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

## 因果注意力模块实现

In [2]:
class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # regularization
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.dropout = config.dropout
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            # manual implementation of attention
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y


## LayerNorm实现
使用的是`torch.nn.functional.layer_norm`中的`layer_norm`方法实现，

In [3]:
class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

## MLP实现
需要注意，激活函数使用的是`GELU`

In [4]:
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu    = nn.GELU()
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

## DecoderLayer实现
一个DecoderLayer包含一个`CausalSelfAttention`和`MLP`，后面分别都接一个`LayerNorm`。下面的代码实现中`DecoderLayer`命名为`Block`。

In [5]:
class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

## GPTConfig
定义了GPT的模型参数

In [6]:
@dataclass
class GPTConfig:
    block_size: int = 1024 # block_size表示的意思是模型处理的最大序列长度，默认为1024
    vocab_size: int = 50304 # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    dropout: float = 0.0
    bias: bool = True # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster

## GPT模型

In [7]:
class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd), 
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = LayerNorm(config.n_embd, bias=config.bias),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        # with weight tying when using torch.compile() some warnings get generated:
        # "UserWarning: functional_call was passed multiple values for tied weights.
        # This behavior is deprecated and will be an error in future versions"
        # not 100% sure what this is, so far seems to be harmless. TODO investigate
        self.transformer.wte.weight = self.lm_head.weight # https://paperswithcode.com/method/weight-tying

        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters
        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        """
        Return the number of parameters in the model.
        For non-embedding count (default), the position embeddings get subtracted.
        The token embeddings would too, except due to the parameter sharing these
        params are actually used as weights in the final layer, so we include them.
        """
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            loss = None

        return logits, loss

    def crop_block_size(self, block_size):
        # model surgery to decrease the block size if necessary
        # e.g. we may load the GPT2 pretrained model checkpoint (block size 1024)
        # but want to use a smaller block size for some smaller, simpler model
        assert block_size <= self.config.block_size
        self.config.block_size = block_size
        self.transformer.wpe.weight = nn.Parameter(self.transformer.wpe.weight[:block_size])
        for block in self.transformer.h:
            if hasattr(block.attn, 'bias'):
                block.attn.bias = block.attn.bias[:,:,:block_size,:block_size]

    @classmethod
    def from_pretrained(cls, model_type, override_args=None):
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        override_args = override_args or {} # default to empty dict
        # only dropout can be overridden see more notes below
        assert all(k == 'dropout' for k in override_args)
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        print("forcing vocab_size=50257, block_size=1024, bias=True")
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        config_args['bias'] = True # always True for GPT model checkpoints
        # we can override the dropout rate, if desired
        if 'dropout' in override_args:
            print(f"overriding dropout rate to {override_args['dropout']}")
            config_args['dropout'] = override_args['dropout']
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")

        return optimizer

    def estimate_mfu(self, fwdbwd_per_iter, dt):
        """ estimate model flops utilization (MFU) in units of A100 bfloat16 peak FLOPS """
        # first estimate the number of flops we do per iteration.
        # see PaLM paper Appendix B as ref: https://arxiv.org/abs/2204.02311
        N = self.get_num_params()
        cfg = self.config
        L, H, Q, T = cfg.n_layer, cfg.n_head, cfg.n_embd//cfg.n_head, cfg.block_size
        flops_per_token = 6*N + 12*L*H*Q*T
        flops_per_fwdbwd = flops_per_token * T
        flops_per_iter = flops_per_fwdbwd * fwdbwd_per_iter
        # express our flops throughput as ratio of A100 bfloat16 peak flops
        flops_achieved = flops_per_iter * (1.0/dt) # per second
        flops_promised = 312e12 # A100 GPU bfloat16 peak flops is 312 TFLOPS
        mfu = flops_achieved / flops_promised
        return mfu

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx


# 一、config

In [8]:
out_dir = 'out-test'
eval_interval = 500
log_interval = 4
eval_iters = 10
always_save_checkpoint = False # if True, always save a checkpoint after each eval
init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'
# wandb logging
wandb_log = False # disabled by default
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt' # 'run' + str(time.time())
# data
dataset = 'shakespeare_char'
gradient_accumulation_steps = 4 # used to simulate larger batch sizes
batch_size = 16 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 256  #text
# 我很开心给
# 大家做饭<eos>

# model
# n_layer = 2
# n_head = 4
# n_embd = 128
n_layer = 12
n_head = 12
n_embd = 768
dropout = 0.1 # for pretraining 0 is good, for finetuning try 0.1+
bias = False # do we use bias inside LayerNorm and Linear layers?
# adamw optimizer
learning_rate = 1e-3 # max learning rate
max_iters = 5000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.99
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 100 # how many steps to warm up for
lr_decay_iters = 5000 # should be ~= max_iters per Chinchilla
min_lr = 1e-4 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# DDP settings
backend = 'nccl' # 'nccl', 'gloo', etc.
# system
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = True # use PyTorch 2.0 to compile the model to be faster
# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
# exec(open('configurator.py').read()) # overrides from command line or config file
config = {k: globals()[k] for k in config_keys} # will be useful for logging

/home/libinbin/anaconda3/envs/gpt/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647382455/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


下面是设置基本允许环境：
- PyTorch中的`manual_seed`函数用于设置生成随机数的种子，以确保结果的**可重复性**。这在需要一致结果的实验中特别有用
- `allow_tf32`的作用是是否允许`PyTorch`在内部使用`TensorFloat32`（TF32）的 **tensor core** (在NVIDIA GPU的新的Ampere架构开始使用) 来计算`matmul`（矩阵乘法和分批矩阵乘法）和`卷积`
- `nullcontext`是一个非常简单的上下文管理器，它不执行任何操作。它的主要用途是在需要动态决定是否需要使用上下文管理器时作为一个占位符使用。

In [9]:
seed_offset = 0
# os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
# device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
device_type = 'cpu'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

## 加载数据

下载数据：允许`./data/download.py`，下载`Shakespeare`数据集，并对数据进行预处理。按照`character-level`进行建模，所以这里不适用`BPE`进行编码， 而是直接将字符映射为整型。 将保存包含`id`的`train.bin`和`value.bin`，`meta.bin`包含`encoder`和`decoder`以及其他一些相关信息。

In [10]:
# data
dataset = 'shakespeare_char'
data_dir = os.path.join('data', dataset)
data_dir

'data/shakespeare_char'

In [11]:
meta_path = os.path.join(data_dir, 'meta.pkl')
meta_vocab_size = None
if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    meta_vocab_size = meta['vocab_size']
    print(f"vocab_size = {meta_vocab_size} (inside {meta_path})")

vocab_size = 65 (inside data/shakespeare_char/meta.pkl)


# 2. Load Model

## 生成测试数据 
下面随机生成一个`[16,256]`维度的张量`X`和`Y`。

注意：张量`X`和`Y`表示的是对输入文本进行`tokenize`分词之后得到的张量，其中的每个数字代表的`token id`，所以，张量`X`和`Y`中的每个元素的取值范围必须是`[0, vocab_size]`，在这里就是`[0,65]`。

`[16,256]`的意思是16个batch的数据，每个batch中token长度为256，也就是`[batch_size, token_num]`

In [12]:
device = 'cpu'
X = torch.randint(0,65,(16,256)).to(device)
targets = torch.randint(0,65,(16,256)).to(device)

## 配置测试模型
下面配置一个层数`n_layer=2`, `batch_size=16`, `n_embd=128`的模型，表示`DecoderLayer`只有2层，每个`token`会被 `Embedding`成128维。

In [13]:
# class GPTConfig:
batch_size = 16 
block_size = 1024  # lenght
n_layer = 2
n_head = 4
n_embd = 128

model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=1024,
                bias=bias, vocab_size=None, dropout=dropout) # start with model_args from command line

if meta_vocab_size is None:
    print("defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)")
model_args['vocab_size'] = meta_vocab_size if meta_vocab_size is not None else 50304
gpt_config = GPTConfig(**model_args)
model = GPT(gpt_config).to(device)
print(model)

number of parameters: 0.40M
GPT(
  (transformer): ModuleDict(
    (wte): Embedding(65, 128)
    (wpe): Embedding(1024, 128)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-1): 2 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=128, out_features=384, bias=False)
          (c_proj): Linear(in_features=128, out_features=128, bias=False)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=128, out_features=512, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=512, out_features=128, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=128, out_features=65, bias=False)
)


## 进行一次前向传播
下面使用生成的张量`X`进行一次前向传播计算。会逐步打印出前向计算的结果，可以看到数据在前向计算过程中的变化。

前向推理的过程就是`GPT`模型中`forward`函数的执行过程，下面也是对`forward`函数的逐行解读。

In [14]:
print(model.transformer.wpe)
print(X.shape)

Embedding(1024, 128)
torch.Size([16, 256])


下面先使用输入`X`进行一次`token embedding`计算，得到的维度为`[16, 256, 128]`，即`[batch_size, token_num, embedding_size]`，每个`token`编码成128维。然后计算每个token的位置信息，这里使用最简单的位置信息计算方式，直接将序号作为位置信息`pos`，通过`model.transformer.wpe`将位置信息`embedding`之后，加入到`token_embedding`当中。

建议细看一下上面`GPT`类中的`forward`函数。

In [15]:
batch_size, token_num = X.size()
# forward the GPT model itself
token_embedding = model.transformer.wte(X) # token embeddings of shape (b, t, n_embd)
print('token_embedding:', token_embedding.shape)

pos = torch.arange(0, token_num, dtype=torch.long) # shape (t)
pos_embedding = model.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
print('词嵌入向量维n_embd = ', n_embd)
print('tok_emb:', token_embedding.shape)
print('pos_emb:', pos_embedding.shape)
x = model.transformer.drop(token_embedding + pos_embedding)
print('Embedding完成之后的输入:', x.shape)

token_embedding: torch.Size([16, 256, 128])
词嵌入向量维n_embd =  128
tok_emb: torch.Size([16, 256, 128])
pos_emb: torch.Size([256, 128])
Embedding完成之后的输入: torch.Size([16, 256, 128])


下面来看在`DecoderLayer`中的计算过程，也就是`CausalSelfAttention`和`MLP`部分。这里定义的模型是只有2层的Decoder模型，直接输出` model.transformer.h`可以看到模型结构。

In [16]:
print('decoder layers:', model.transformer.h)
for i, block in enumerate(model.transformer.h):
    x = block(x)
    print(f'layer:{i}, decoder x: {x.shape}')


decoder layers: ModuleList(
  (0-1): 2 x Block(
    (ln_1): LayerNorm()
    (attn): CausalSelfAttention(
      (c_attn): Linear(in_features=128, out_features=384, bias=False)
      (c_proj): Linear(in_features=128, out_features=128, bias=False)
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm()
    (mlp): MLP(
      (c_fc): Linear(in_features=128, out_features=512, bias=False)
      (gelu): GELU(approximate='none')
      (c_proj): Linear(in_features=512, out_features=128, bias=False)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)
layer:0, decoder x: torch.Size([16, 256, 128])
layer:1, decoder x: torch.Size([16, 256, 128])


经过两次`DecoderLayer`计算之后，还需要经过一次全连接层得到最终的模型输出，这就是`model.transformer.ln_f`部分。

In [17]:
x = model.transformer.ln_f(x)
print('ln_f x:', x.shape)

ln_f x: torch.Size([16, 256, 128])


可以看到，经过了最后一次的全连接层之后，输出的维度依旧是`[16, 256, 128]`，也就是每个token依旧是编码成128来表示，但是模型真正应该输出的是每个token的`token id`，也就是输出维度为`[16, 256]`，所以这里还需要进行一次解码计算，将128维的`token embedding`向量，解码成`token id`。这就是`GPT`模型中`lm_head`的作用。


### lm_head
`lm_head`与`model.transformer.wte`一样，都是一个线性的Embedding层，用来从**隐藏状态**预测词汇表上的概率分布。并且`lm_head`与`model.transformer.wte`的权重是共享的，是共用一套权重，称为权重共享技术（Weight Tying）。

为什么要将两者的权重进行共享，即`model.transformer.wte.weight = model.lm_head.weight`？
1. `model.transformer.wte.weight`是词嵌入层的权重矩阵，用于将输入的`token`索引编码到相应的向量表示。而`model.lm_head.weight`是解码层的权重矩阵。在GPT以及其他一些Transformer-based模型中，常常采用“权重共享”（Weight Tying）的技术，即将词嵌入层的权重与解码层的权重共享，理由如下：

- **减少参数数量**：共享权重可以降低模型参数总数，特别是在词汇表很大的情况下，显著减小模型的规模。
- **正则化效果**：权重绑带可以被视为一种隐含的正则化技术，它限制了解码器在学习新的映射时的自由度，鼓励模型在嵌入空间中学习到更通用的表示。
- **理论上的一致性**：在语言模型中，词嵌入应该反映词在语境中的含义和用法，而解码器的任务正是基于先前的上下文预测下一个可能出现的词。让这两个层共享权重有助于模型在学习词嵌入时同时考虑到生成任务的需求。

在前向计算过程中，如果给出了`targets`，则说明是训练模式，需要计算loss。否则是推理模式，不需要计算loss。

注意：`lm_head`的计算结果`logits`的维度为`[16, 256, 65]`，说明`lm_head`是将每个token的embedding表示，映射到词表中的概率分布，对于一个张量`[1,256]`，计算完`lm_head`之后为`[1,65]`，其中概率最大的tokekn就是这个最终预测的结果。

In [18]:
if targets is not None:
    # if we are given some desired targets also calculate the loss
    logits = model.lm_head(x)
    loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
    print(loss)
else:
    # inference-time mini-optimization: only forward the lm_head on the very last position
    logits = model.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
    loss = None
print('logits:', logits.shape)

tensor(4.2190, grad_fn=<NllLossBackward0>)
logits: torch.Size([16, 256, 65])


## 在线推理过程generate
`forward`函数在训练和推理过程中都会用到，但是`forward`函数最后经过`lm_head`后，得到的是词表的概率分布，输出维度为`[16, 256, 65]`，而不是一串连续的解码完成的文本。生成字符输出则主要是在`generate`函数中完成。

接下来重点讲的就是`generate`函数。

In [19]:
def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
    """
    Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
    the sequence max_new_tokens times, feeding the predictions back into the model each time.
    Most likely you'll want to make sure to be in model.eval() mode of operation for this.
    """
    for _ in range(max_new_tokens):
        # if the sequence context is growing too long we must crop it at block_size
        idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
        # forward the model to get the logits for the index in the sequence
        logits, _ = self(idx_cond)
        # pluck the logits at the final step and scale by desired temperature
        logits = logits[:, -1, :] / temperature
        # optionally crop the logits to only the top k options
        if top_k is not None:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:, [-1]]] = -float('Inf')
        # apply softmax to convert logits to (normalized) probabilities
        probs = F.softmax(logits, dim=-1)
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1)
        # append sampled index to the running sequence and continue
        idx = torch.cat((idx, idx_next), dim=1)

    return idx

过程解读如下：
1. idx输入（维度为`[batch_size, token_num]`）模型，进行一次前向传播，得到下一个词在词表中的概率分布。
2. 使用`temperature`对最新生成的概率分布进行缩放
3. 使用`top_k`，只取`top_k`个概率分布，其余的设为`-inf`
4. 经过一次`softmax`，将概率归一化到`[0-1]`之间。
5. `torch.multinomial`函数是用于根据概率分布进行采样的工具。它接受一个包含概率的张量作为输入，并返回一个张量，其中每行包含从相应输入行的多项式概率分布中采样的 `num_samples` 个索引。
6. 和输入拼接，然后返回

`torch.multinomial`函数的解读见：[torch.multinomial](../../1-LLM-base-fundamentals/1.6-Pytorch常用函数/torch_multinomial.ipynb)

下面逐步来看运行过程。

In [31]:
# Greedy Generation
idx = X
print("预测词表大小", model.config.vocab_size) # BPE词表
print("输入X:", idx)
print("输入X长度:", idx.shape)
temperature = 0.95
top_k = 10
for _ in range(2):
    logits, _ = model(X)
    print('输出Logits:', logits.shape)   
    print(f"temperature缩放前：{logits[0, -1, :10]}")
    logits = logits[:, -1, :] / temperature
    print('输出Logits:', logits.shape) 
    print(f"temperature缩放后：{logits[0, :10]}")
    if top_k is not None:
        v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
        logits[logits < v[:, [-1]]] = -float('Inf')
    print(f"保留top_k个之后：{logits[0, :]}")
        
    probs = F.softmax(logits, dim=-1)
    print('输出概率分布:', probs.shape)
    # idx_next = torch.argmax(probs , dim=-1)  # 这里取概率最大的作为输出
    idx_next = torch.multinomial(probs, num_samples=1)
    print('输出下一个Token:', idx_next.shape)
    print(idx_next)
    idx = torch.cat((idx, idx_next), dim=1)
    # idx = idx_next
    print("输出token长度:", len(idx[0]))
    print("输出token序列:", idx)
    
    

预测词表大小 65
输入X: tensor([[35, 22,  7,  ...,  1, 34, 13],
        [ 5,  8,  6,  ..., 60, 14, 62],
        [44, 29, 53,  ..., 59, 23, 63],
        ...,
        [ 5,  4, 63,  ..., 42, 59, 37],
        [33, 55, 28,  ...,  3, 21, 25],
        [ 0, 39, 17,  ..., 40, 61, 32]])
输入X长度: torch.Size([16, 256])
输出Logits: torch.Size([16, 1, 65])
temperature缩放前：tensor([-0.0529, -0.2387,  0.3881, -0.1474,  0.0090,  0.0731, -0.6404, -0.4169,
        -0.3666, -0.1400], grad_fn=<SliceBackward0>)
输出Logits: torch.Size([16, 65])
temperature缩放后：tensor([-0.0556, -0.2512,  0.4086, -0.1551,  0.0094,  0.0769, -0.6741, -0.4388,
        -0.3859, -0.1473], grad_fn=<SliceBackward0>)
保留top_k个之后：tensor([  -inf,   -inf, 0.4086,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf,
          -inf,   -inf, 0.4623,   -inf, 1.1983,   -inf,   -inf,   -inf, 0.3870,
        0.2944,   -inf,   -inf,   -inf,   -inf,   -inf, 0.3626,   -inf,   -inf,
          -inf,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf,
          

思考：为什么`Temperature`越大，生成的结果越随机：

`Softmax`公式为：
$$softmax(z_i)=\frac{\epsilon^{z_i / T}}{\sum_j \epsilon^{z_j / T}}$$
在这个修改后的公式中：
- 当 T = 1 时，它就是普通的 softmax 函数。
- 当 T > 1 时，分数之间的差异会减小，导致概率分布更加均匀（增加随机性和创新性），T越大，概率差异越小。
- 当 T < 1 时，分数之间的差异会增大，使得高分数的词的概率远远高于低分数的词（增加确定性和预测性）。

## 使用Huggingface权重
下面使用Huggingface中的`gpt2`权重来进行推理。模型下载：[openai-community/gpt2](https://huggingface.co/openai-community/gpt2)

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
